In [1]:

#--------------------------------------------------------------------
# Exercises 4.

# Exercise 4.1.

# Titanic passengers data – 1310 observations and 15 variables:
# passenger_id – Unique passenger id
# pclass – Ticket class (1 = 1st, 2 = 2nd, 3 = 3rd)
# survived – Survival (0 = No, 1 = Yes)
# name – Name and SUrname
# sex – Sex (0 = Male, 1 = Female)
# age – Age in years
# sibsp – # of siblings / spouses aboard the Titanic
# parch – # of parents / children aboard the Titanic
# ticket – Ticket number
# fare – Passenger fare
# cabin – Cabin number
# embarked – Port of Embarkation (C = Cherbourg, Q = Queenstown, S = Southampton)
# boat – Lifeboat (if survived)
# body – Body number (if did not survive and body was recovered)
# home.dest – Home/Destination
# 
# Re-run your best models for all algorithms for 5-fold CV. 
# Check the stability of results for repeated K-fold
# Check in repeated k-fold CV if adding stratification changes your results (stability)
# Check if you didnt overfit in your models. Check if you can imrpove you validation score.

In [2]:
# Imports
import pandas as pd
import numpy as np
import pickle
import statsmodels.api as sm
import matplotlib.pyplot as plt
import gc
from sklearn.metrics import roc_auc_score

plt.style.use('seaborn-v0_8-ticks')
%matplotlib inline

In [3]:
titanic = pd.read_csv("../data/titanic.csv")
print(titanic.count())
print(titanic.isnull().sum())
# titanic.head()

passenger_id    1309
pclass          1309
survived        1309
name            1309
sex             1309
age             1046
sibsp           1309
parch           1309
ticket          1309
fare            1308
cabin            295
embarked        1307
boat             486
body             121
home.dest        745
dtype: int64
passenger_id       0
pclass             0
survived           0
name               0
sex                0
age              263
sibsp              0
parch              0
ticket             0
fare               1
cabin           1014
embarked           2
boat             823
body            1188
home.dest        564
dtype: int64


In [4]:
# Drop col boat, body, home.dest, carbin due to high number of missing values
titanic = titanic.drop(columns=['boat', 'body', 'home.dest', 'cabin'])
# fill missing age, fares values with median
titanic['age'] = titanic['age'].fillna(titanic['age'].median())
titanic['fare'] = titanic['fare'].fillna(titanic['fare'].median())
# fill missing embarked values with mode
titanic['embarked'] = titanic['embarked'].fillna(titanic['embarked'].mode()[0])
# check for any remaining missing values
print(titanic.isnull().sum())

passenger_id    0
pclass          0
survived        0
name            0
sex             0
age             0
sibsp           0
parch           0
ticket          0
fare            0
embarked        0
dtype: int64


In [5]:
# OLS Regression
import statsmodels.api as smf
X = titanic[['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare']]
y = titanic['survived']
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
predictions = model.predict(X)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               survived   R-squared:                       0.365
Model:                            OLS   Adj. R-squared:                  0.362
Method:                 Least Squares   F-statistic:                     124.8
Date:                Mon, 12 Jan 2026   Prob (F-statistic):          9.84e-125
Time:                        09:04:11   Log-Likelihood:                -615.15
No. Observations:                1309   AIC:                             1244.
Df Residuals:                    1302   BIC:                             1281.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7530      0.059     12.713      0.000       0.637       0.869
pclass        -0.1615      0.017     -9.687      0.000      -0.194      -0.129
sex            0.4979      0.023     21.363      0.000       0.452       0.544
age           -0.0056      0.001     -6.135      0.000      -0.007      -0.004
sibsp         -0.0450      0.011     -3.960      0.000      -0.067      -0.023
parch         -0.0050      0.014     -0.358      0.720      -0.032       0.022
fare           0.0004      0.000      1.487      0.137      -0.000       0.001
==============================================================================
Omnibus:                       54.722   Durbin-Watson:                   1.801
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               61.045
Skew:                           0.529   Prob(JB):                     5.55e-14
Kurtosis:                       2.986   Cond. No.                         368.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [6]:
cols = ["survived", "pclass", "sex", "age", "sibsp"]
titanic_clean = titanic.dropna(subset=cols)

mod = sm.GLM.from_formula(
    formula="survived ~ pclass + sex + age + sibsp",
    data=titanic_clean,
    family=sm.families.Binomial()
)
res = mod.fit()
print(res.summary())
y_test = titanic_clean.survived
preds = res.predict(titanic_clean)
roc_auc_score(y_test, preds)

                 Generalized Linear Model Regression Results                  
Dep. Variable:               survived   No. Observations:                 1309
Model:                            GLM   Df Residuals:                     1304
Model Family:                Binomial   Df Model:                            4
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -606.21
Date:                Mon, 12 Jan 2026   Deviance:                       1212.4
Time:                        09:04:11   Pearson chi2:                 1.34e+03
No. Iterations:                     5   Pseudo R-squ. (CS):             0.3322
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.1070      0.345      6.112      0.0

0.839639060568603

In [7]:
# Re-run your best models for all algorithms for 5-fold CV
from sklearn.model_selection import KFold
scores = []
kf = KFold(n_splits=5, shuffle=True, random_state=np.random.randint(0, 10000))
for train, test in kf.split(titanic_clean.index.values):
    mod = sm.GLM.from_formula(
        formula="survived ~ pclass + sex + age + sibsp + parch + fare",
        data=titanic_clean.iloc[train],
        family=sm.families.Binomial()
    )
    res = mod.fit()
    predsTrain = res.predict(titanic_clean.iloc[train])
    preds = res.predict(titanic_clean.iloc[test])
    train_auc = roc_auc_score(titanic_clean.iloc[train].survived, predsTrain)
    val_auc = roc_auc_score(titanic_clean.iloc[test].survived, preds)
    scores.append(val_auc)  # add this line
    print("Train AUC:", round(train_auc, 4), "Valid AUC:", round(val_auc, 4))

print(f'5-Fold CV AUC: Mean = {np.mean(scores):.4f}, Std = {np.std(scores):.4f}')

Train AUC: 0.8412 Valid AUC: 0.8371
Train AUC: 0.8491 Valid AUC: 0.8057
Train AUC: 0.8372 Valid AUC: 0.854
Train AUC: 0.8339 Valid AUC: 0.8648
Train AUC: 0.8423 Valid AUC: 0.8328
5-Fold CV AUC: Mean = 0.8389, Std = 0.0202


In [8]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import RepeatedKFold, RepeatedStratifiedKFold
from sklearn.metrics import roc_auc_score

# FORMULA 
formula = "survived ~ pclass + sex + age + sibsp + parch + fare"

def run_cv(cv_strategy, data):
    scores_val = []
    scores_train = []
    
    # cv_strategy.split cần (X, y) để thực hiện stratification
    for train_idx, test_idx in cv_strategy.split(data, data['survived']):
        train_df = data.iloc[train_idx]
        test_df = data.iloc[test_idx]
        
        # Huấn luyện mô hình Logistic (GLM Binomial)
        model = sm.GLM.from_formula(formula=formula, data=train_df, family=sm.families.Binomial())
        res = model.fit()
        
        # Dự đoán
        train_preds = res.predict(train_df)
        test_preds = res.predict(test_df)
        
        # Tính AUC
        train_auc = roc_auc_score(train_df['survived'], train_preds)
        val_auc = roc_auc_score(test_df['survived'], test_preds)
        
        scores_train.append(train_auc)
        scores_val.append(val_auc)
        
    return scores_train, scores_val


# 1. Repeated K-Fold (5-fold, lặp lại 10 lần)
rkf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=42)
train_rkf, val_rkf = run_cv(rkf, titanic_clean)

# 2. Repeated Stratified K-Fold (5-fold, lặp lại 10 lần) - CÓ PHÂN TẦNG
rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)
train_rskf, val_rskf = run_cv(rskf, titanic_clean)

# --- IN KẾT QUẢ SO SÁNH ---
print(f"{'Method':<30} | {'Mean AUC':<10} | {'Std Dev':<10}")
print("-" * 55)
print(f"{'Repeated K-Fold (Valid)':<30} | {np.mean(val_rkf):.4f} | {np.std(val_rkf):.4f}")
print(f"{'Repeated Stratified (Valid)':<30} | {np.mean(val_rskf):.4f} | {np.std(val_rskf):.4f}")
print(f"{'Repeated Stratified (Train)':<30} | {np.mean(train_rskf):.4f} | {np.std(train_rskf):.4f}")

Method                         | Mean AUC   | Std Dev   
-------------------------------------------------------
Repeated K-Fold (Valid)        | 0.8380 | 0.0231
Repeated Stratified (Valid)    | 0.8376 | 0.0223
Repeated Stratified (Train)    | 0.8409 | 0.0055


In [9]:

# Exercise 4.2.
# Wine Quality Data Set: "data/wines.csv"
# source: https://archive.ics.uci.edu/ml/datasets/wine+quality
# The file contains data on samples of white and red Portuguese wine 
# Vinho Verde. 
# Various physico-chemical characteristics of individual samples
# are available as well as wine quality scores on a point scale (0-10) 
# made by specialists.

# Re-run your best models for all algorithms for 5-fold CV. 
# Check the stability of results for repeated K-fold
# Check in repeated k-fold CV if adding stratification changes your results (stability).
# Compare the effect of stratification with titanic problem.
# Check if you didnt overfit in your models. Check if you can imrpove you validation score.

In [10]:
wines = pd.read_csv("../data/wines.csv")
wines.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,type,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,red,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,red,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,red,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,red,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,red,5


In [11]:
import statsmodels.formula.api as smf
from sklearn.metrics import r2_score

# Quality regression
mod = smf.ols(
    formula="quality ~ fixed_acidity + volatile_acidity + citric_acid + residual_sugar + chlorides + free_sulfur_dioxide + total_sulfur_dioxide + density + pH + sulphates + alcohol",
    data=wines
)
res = mod.fit()
print(res.summary())
y_test = wines["quality"]
preds = res.predict(wines)
print(f'R2 score: {r2_score(y_test, preds):.4f}')

                            OLS Regression Results                            
Dep. Variable:                quality   R-squared:                       0.292
Model:                            OLS   Adj. R-squared:                  0.291
Method:                 Least Squares   F-statistic:                     243.3
Date:                Mon, 12 Jan 2026   Prob (F-statistic):               0.00
Time:                        09:04:21   Log-Likelihood:                -7215.5
No. Observations:                6497   AIC:                         1.445e+04
Df Residuals:                    6485   BIC:                         1.454e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               55.7627 

In [12]:
# Drop chlorides, citric_acid column
new_wines = wines.drop(columns=["chlorides", "citric_acid"])
mod = smf.ols(
    formula="quality ~ fixed_acidity + volatile_acidity + residual_sugar + free_sulfur_dioxide + total_sulfur_dioxide + density + pH + sulphates + alcohol",
    data=new_wines
)
res = mod.fit()
print(res.summary())
y_test = new_wines["quality"]
preds = res.predict(new_wines)
print(f'R2 score: {r2_score(y_test, preds):.4f}')

                            OLS Regression Results                            
Dep. Variable:                quality   R-squared:                       0.292
Model:                            OLS   Adj. R-squared:                  0.291
Method:                 Least Squares   F-statistic:                     296.7
Date:                Mon, 12 Jan 2026   Prob (F-statistic):               0.00
Time:                        09:04:33   Log-Likelihood:                -7217.8
No. Observations:                6497   AIC:                         1.446e+04
Df Residuals:                    6487   BIC:                         1.452e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               60.0409 

In [17]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from sklearn.model_selection import KFold, RepeatedKFold
from sklearn.metrics import r2_score

# 1. Định nghĩa công thức (đã loại bỏ chlorides và citric_acid)
formula = "quality ~ fixed_acidity + volatile_acidity + residual_sugar + free_sulfur_dioxide + total_sulfur_dioxide + density + pH + sulphates + alcohol"

def run_wine_cv(cv_strategy, data):
    scores_val = []
    scores_train = []
    
    # Sửa lỗi: Truyền thêm data['quality'] vào split() để StratifiedKFold hoạt động
    for train_idx, test_idx in cv_strategy.split(data, data['quality']):
        train_df = data.iloc[train_idx]
        test_df = data.iloc[test_idx]
        
        model = smf.ols(formula=formula, data=train_df).fit()
        
        train_preds = model.predict(train_df)
        test_preds = model.predict(test_df)
        
        train_r2 = r2_score(train_df['quality'], train_preds)
        val_r2 = r2_score(test_df['quality'], test_preds)
        
        scores_train.append(train_r2)
        scores_val.append(val_r2)
        
    return scores_train, scores_val

# Sau đó chạy lại lệnh cũ:
rskf_wine = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)
train_r2_strat, val_r2_strat = run_wine_cv(rskf_wine, new_wines)

print(f"Stratified Mean R2: {np.mean(val_r2_strat):.4f}, Std: {np.std(val_r2_strat):.4f}")

# --- THỰC HIỆN ---

# 1. 5-Fold CV thông thường
kf = KFold(n_splits=5, shuffle=True, random_state=42)
train_r2_kf, val_r2_kf = run_wine_cv(kf, new_wines)

# 2. Repeated K-Fold (5-fold, lặp lại 10 lần) để kiểm tra tính ổn định (Stability)
rkf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=42)
train_r2_rkf, val_r2_rkf = run_wine_cv(rkf, new_wines)

# --- IN KẾT QUẢ ---
print(f"--- 5-Fold Cross Validation ---")
print(f"Mean R2 (Valid): {np.mean(val_r2_kf):.4f}")
print(f"Std R2 (Stability): {np.std(val_r2_kf):.4f}")

print(f"\n--- Repeated K-Fold (10 repeats) ---")
print(f"Mean R2 (Valid): {np.mean(val_r2_rkf):.4f}")
print(f"Std R2 (Stability): {np.std(val_r2_rkf):.4f}")
print(f"Mean R2 (Train): {np.mean(train_r2_rkf):.4f}")

Stratified Mean R2: 0.2885, Std: 0.0187
--- 5-Fold Cross Validation ---
Mean R2 (Valid): 0.2879
Std R2 (Stability): 0.0159

--- Repeated K-Fold (10 repeats) ---
Mean R2 (Valid): 0.2880
Std R2 (Stability): 0.0167
Mean R2 (Train): 0.2919


In [18]:
from sklearn.model_selection import RepeatedStratifiedKFold

# Chạy thử Stratified cho Wine (dùng quality làm biến phân tầng)
rskf_wine = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)
train_r2_strat, val_r2_strat = run_wine_cv(rskf_wine, new_wines)

print(f"Stratified Mean R2: {np.mean(val_r2_strat):.4f}, Std: {np.std(val_r2_strat):.4f}")

Stratified Mean R2: 0.2885, Std: 0.0187
